In [1]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import dates as mpl_dates
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta
import pytz
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import pearsonr
random.seed(123)
import requests
import os




In [2]:
# data_dictionary = dict()
def authenticate_and_request(APITocken, APPType, request_body):
    # API endpoint URL (replace with the actual API endpoint)
    api_url = 'https://echt.seelive.io/API/DataAI/API_GetSensorData'

    # Construct headers with authentication tokens
    headers = {
        'Authorization': f'Bearer {APITocken}',
        'APITocken': APITocken,
        'APPType': APPType
    }

    try:
        # Send POST request to the API with request body
        response = requests.post(api_url, headers=headers, json=request_body)

        # Check for successful response (status code 200)
        if response.status_code == 200:
            data = response.json()  # Assuming the API returns JSON data
            print('API Response:')
            return data
#             print(data)
#             return data_dictionary.update(data)
        else:
            print(f'Error: HTTP Status Code {response.status_code}')
            print(response.text)  # Print the response content for debugging

    except Exception as e:
        print(f'Error: {e}')

In [3]:
ug_data_list = []
for i in ['ENE00933', 'ENE00960', 'ENE00950']:
    APITocken='Rth-0987u-wert-3456'
    APPType='AIUSER'

    # Define the start and end date strings------------------------------------------------------
    start_date = datetime(2023, 9, 30, 0, 0)
    end_date = datetime(2023, 11, 20, 23, 59)

    # Define a custom datetime format
    custom_format = "%d-%b-%Y %I:%M %p"


    # Format the datetime objects using the custom format
    start_date_str = start_date.strftime(custom_format)
    end_date_str = end_date.strftime(custom_format)

    print(start_date_str)
    print(end_date_str)

    request_body =  {
        "SensorID":i,
         "FromDate":start_date_str,
         "ToDate":end_date_str,
         "DataInteval":1,
         "DataType":"R"
    }

    data = authenticate_and_request(APITocken, APPType, request_body)
    
    df = pd.DataFrame(data['SearchDetail'])
    
    # Drop columns with all zeroes
    New_df_one = df.loc[:, (df != 0).any(axis=0)]
    ug_data_list.append(New_df_one)

30-Sep-2023 12:00 AM
20-Nov-2023 11:59 PM
API Response:
30-Sep-2023 12:00 AM
20-Nov-2023 11:59 PM
API Response:
30-Sep-2023 12:00 AM
20-Nov-2023 11:59 PM
API Response:


In [4]:
ug_data_list[0].head()

,DataDate,DeviceID,PM2_5,PM10,SOX,NOX,Temp,Hum
0,2023-09-30T05:30:00,ENE00933,24.0,27.0,75.16,37.37,21.3,99.58
1,2023-09-30T05:31:00,ENE00933,21.0,23.0,75.96,38.04,21.3,99.40
2,2023-09-30T05:32:00,ENE00933,22.0,24.0,74.66,37.42,21.3,99.16
3,2023-09-30T05:33:00,ENE00933,21.0,23.0,74.69,38.20,21.3,99.81
4,2023-09-30T05:34:00,ENE00933,22.0,24.0,74.55,39.02,21.4,99.49


In [5]:
ug_data_list[1].head()

,DataDate,DeviceID,PM2_5,PM10,O3,CO,Temp,Hum,ITemp,CO2,AQI
0,2023-09-30T05:30:00,ENE00960,18.0,20.0,0.01,0.01,21.4,97.0,32.75,11379.0,20.0
1,2023-09-30T05:31:00,ENE00960,18.0,20.0,0.01,0.01,21.4,97.0,32.75,11379.0,22.0
2,2023-09-30T05:32:00,ENE00960,15.0,17.0,0.01,0.02,21.4,97.0,32.75,11378.0,16.0
3,2023-09-30T05:33:00,ENE00960,17.0,19.0,0.02,0.01,21.4,97.0,32.75,11378.0,20.0
4,2023-09-30T05:34:00,ENE00960,17.0,19.0,0.01,0.01,21.4,97.0,32.75,11374.0,19.0


In [6]:
ug_data_list[2].head()

,DataDate,DeviceID,PM2_5,PM10,SOX,NOX,Temp,Hum
0,2023-09-30T05:46:00,ENE00950,21.0,23.0,78.99,40.53,21.8,97.0
1,2023-09-30T06:58:00,ENE00950,21.0,23.0,83.24,37.89,21.6,100.0
2,2023-09-30T06:59:00,ENE00950,21.0,23.0,83.40,36.61,21.6,100.0
3,2023-09-30T07:00:00,ENE00950,20.0,22.0,83.34,38.14,21.6,100.0
4,2023-09-30T07:01:00,ENE00950,19.0,21.0,82.39,37.00,21.6,100.0


In [7]:
for i in ug_data_list:
    columns_to_drop = ['SOX', 'NOX', 'Temp', 'Hum', 'O3', 'CO', 'ITemp', 'CO2', 'AQI']
    
    # Check if columns exist before dropping them
    columns_to_drop_existing = [col for col in columns_to_drop if col in i.columns]
    i.drop(columns=columns_to_drop_existing, inplace=True)  # Drop the column if it exists


In [8]:
ug_data_list[0]

,DataDate,DeviceID,PM2_5,PM10
0,2023-09-30T05:30:00,ENE00933,24.0,27.0
1,2023-09-30T05:31:00,ENE00933,21.0,23.0
2,2023-09-30T05:32:00,ENE00933,22.0,24.0
3,2023-09-30T05:33:00,ENE00933,21.0,23.0
4,2023-09-30T05:34:00,ENE00933,22.0,24.0
...,...,...,...,...
56885,2023-11-21T05:24:00,ENE00933,23.0,28.0
56886,2023-11-21T05:25:00,ENE00933,22.0,26.0
56887,2023-11-21T05:26:00,ENE00933,21.0,23.0
56888,2023-11-21T05:27:00,ENE00933,23.0,27.0


In [9]:
ug_data_list[1]

,DataDate,DeviceID,PM2_5,PM10
0,2023-09-30T05:30:00,ENE00960,18.0,20.0
1,2023-09-30T05:31:00,ENE00960,18.0,20.0
2,2023-09-30T05:32:00,ENE00960,15.0,17.0
3,2023-09-30T05:33:00,ENE00960,17.0,19.0
4,2023-09-30T05:34:00,ENE00960,17.0,19.0
...,...,...,...,...
51784,2023-11-21T05:24:00,ENE00960,20.0,23.0
51785,2023-11-21T05:25:00,ENE00960,17.0,19.0
51786,2023-11-21T05:26:00,ENE00960,18.0,21.0
51787,2023-11-21T05:27:00,ENE00960,17.0,19.0


In [10]:
ug_data_list[2]

,DataDate,DeviceID,PM2_5,PM10
0,2023-09-30T05:46:00,ENE00950,21.0,23.0
1,2023-09-30T06:58:00,ENE00950,21.0,23.0
2,2023-09-30T06:59:00,ENE00950,21.0,23.0
3,2023-09-30T07:00:00,ENE00950,20.0,22.0
4,2023-09-30T07:01:00,ENE00950,19.0,21.0
...,...,...,...,...
56250,2023-11-21T05:24:00,ENE00950,16.0,18.0
56251,2023-11-21T05:25:00,ENE00950,16.0,18.0
56252,2023-11-21T05:26:00,ENE00950,15.0,18.0
56253,2023-11-21T05:27:00,ENE00950,14.0,15.0


In [11]:
ug_data_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56890 entries, 0 to 56889
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DataDate  56890 non-null  object 
 1   DeviceID  56890 non-null  object 
 2   PM2_5     56890 non-null  float64
 3   PM10      56890 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.7+ MB


In [12]:
# # Create a folder if it doesn't exist
# folder_name = 'raw_data_from_API'
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)

# Save each DataFrame to a CSV file with a unique name
for idx, df in enumerate(ug_data_list, start=1):
    sensor_id = df['DeviceID'][0]
    file_name = f'DataFrame_{sensor_id}.csv'
    # df.drop('DeviceID', axis=1, inplace=True)  # Drop the desired column
    df.to_csv(file_name, index=False)